In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [30]:
import dataloader as dl

# dataSource
td, tl, pd, pl = dl.read_bci_data()

In [23]:
x = torch.arange(6)

x = x.view(2, -1)
x

tensor([[0, 1, 2],
        [3, 4, 5]])

In [ ]:
class EEGNET(nn.Module):
  def __init__(self, actFun) -> None:
    match actFun:
      case "elu":
        self.activation = F.elu
      case "relu":
        self.activation = F.relu
      case "leaky_relu":
        self.activation = F.leaky_relu

    # Layer 1
    self.FirstConv = nn.Sequential(
      nn.Conv2d(1, 16, kernal_size=(1, 51), stride=(1, 1), padding=(0, 25), bias=False)
      , nn.BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_status=True)
    )
    
    # Layer 2
    self.DepthWiseConv = nn.Sequential(
      nn.Conv2d(16, 32, kernel_size=(2, 1), stride=(1, 1), groups=16, bias=False)
      , nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_status=True)
      , nn.AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
      , self.activation
      , nn.Dropout(p=0.25)
    )

    # Layer 3
    self.SeperableConv = nn.Sequential(
      nn.Conv2d(32, 32, kernel_size=(1, 15), stride=(1, 1), padding=(0, 7), bias=False)
      , nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_status=True)
      , self.activation
      , nn.AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
      , nn.Dropout(p=0.25)
    )

    self.Classify = nn.Sequential(
      nn.Linear(in_features=736, out_features=2, bias=True)
    )
  
  def foward(self, x):
    x = self.FirstConv(x)
    x = self.DepthWiseConv(x)
    x = self.SeperableConv(x)

    x = x.view(-1, 736) # reshape to fit the classifier
    x = self.Classify(x)

    return x